In [1]:
import sys
!{sys.executable} -m pip3 install pandas
!{sys.executable} -m pip3 install numpy

c:\users\majid\appdata\local\programs\python\python37\python.exe: No module named pip3
c:\users\majid\appdata\local\programs\python\python37\python.exe: No module named pip3


In [2]:
import pandas
import numpy as np

###### Read and show data

In [78]:
iris_data = pandas.read_csv('iris.data', header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
print(iris_data.shape)
iris_data.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


###### KNN: Run KNN algorithm to predict class of test data

In [79]:
def KNN(data, test, k):
    """ This function gets data and test sample and k parameter then predicts class of test according to K nearest data """
    data_without_label = data.drop('class', axis=1)
    distances = np.sqrt(np.square(data_without_label - test).sum(axis=1))  # distance of test to all data
    k_smallest_dist_index = np.argpartition(distances, k)[:k]  # select data index of k smallest distance
    nearest_samples = data.iloc[k_smallest_dist_index]
    all_classes = data['class'].unique()
    weighted_voting = {c:0 for c in all_classes}
    for index, sample in nearest_samples.iterrows():
        weighted_voting[sample['class']] += (1/distances[index])  # use inverse of distance as weightes in voting
    predicted_label = max(weighted_voting, key=lambda x:weighted_voting[x])
    return predicted_label

###### Use KNN algorithm to predict class of all test data and evaluate results

In [80]:
def evaluate_test_data(train_data, test_data, k, confusion_matrix):
    """ This function gets training and testing data and K parameter 
        then use KNN algorithm to predict class of any sample in test data and evaluate results
    """
    truly_predicted = 0
    all_predicted = test_data.shape[0]
    for index, row in test_data.iterrows():
        real_label = row.pop('class')
        predicted_label = KNN(train_data, row, k)
        confusion_matrix[real_label][predicted_label] += 1
        if real_label == predicted_label:
            truly_predicted += 1
    accuracy = truly_predicted/all_predicted
    return accuracy, confusion_matrix

###### Use 10-fold cross velidation to evaluate results

In [81]:
def k_fold_cross_validation(data, k_fold, k_nearest):
    """ This functions gets all data and uses 10-fold cross validation to evaluate KNN results """
    number_of_data = data.shape[0]
    fold_size = int(number_of_data/k_fold)
    all_accuracy = []
    confusion_matrix = {real_class: {predicted_class:0 for predicted_class in iris_data['class'].unique()} 
                        for real_class in iris_data['class'].unique()}
    for i in range(k_fold):
        test_data = data[i*fold_size:(i+1)*fold_size]
        train_data = data.drop(test_data.index)
        accuracy, confusion_matrix = evaluate_test_data(train_data, test_data, k_nearest, confusion_matrix)
        all_accuracy.append(accuracy)
    accuracy_mean = sum(all_accuracy)/len(all_accuracy)
    print('\n##### k-nearest=' + str(k_nearest) + ' #####')
    print('accuracy: ' + str(accuracy_mean))
    print('confusion_matrix: \n' + str(confusion_matrix))
    return accuracy_mean, confusion_matrix

###### Test different K for KNN algorithm to find best one

In [82]:
all_accuracy = []
confusion_matrixes = []
for k_nearest in range(1, 20):
    accuracy, confusion_matrix = k_fold_cross_validation(iris_data, 10, k_nearest)
    all_accuracy.append(accuracy)
    confusion_matrixes.append(confusion_matrix)

c:\users\majid\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.



##### k-nearest=1 #####
accuracy: 0.9600000000000002
confusion_matrix: 
{'Iris-setosa': {'Iris-setosa': 50, 'Iris-versicolor': 0, 'Iris-virginica': 0}, 'Iris-versicolor': {'Iris-setosa': 0, 'Iris-versicolor': 47, 'Iris-virginica': 3}, 'Iris-virginica': {'Iris-setosa': 0, 'Iris-versicolor': 3, 'Iris-virginica': 47}}

##### k-nearest=2 #####
accuracy: 0.9600000000000002
confusion_matrix: 
{'Iris-setosa': {'Iris-setosa': 50, 'Iris-versicolor': 0, 'Iris-virginica': 0}, 'Iris-versicolor': {'Iris-setosa': 0, 'Iris-versicolor': 47, 'Iris-virginica': 3}, 'Iris-virginica': {'Iris-setosa': 0, 'Iris-versicolor': 3, 'Iris-virginica': 47}}

##### k-nearest=3 #####
accuracy: 0.9466666666666669
confusion_matrix: 
{'Iris-setosa': {'Iris-setosa': 50, 'Iris-versicolor': 0, 'Iris-virginica': 0}, 'Iris-versicolor': {'Iris-setosa': 0, 'Iris-versicolor': 46, 'Iris-virginica': 4}, 'Iris-virginica': {'Iris-setosa': 0, 'Iris-versicolor': 4, 'Iris-virginica': 46}}

##### k-nearest=4 #####
accuracy: 0.953333333

###### Show result of KNN with best k

In [84]:
best_accuracy = max(all_accuracy)
best_k = all_accuracy.index(best_accuracy)+1
best_confusion_matrix = confusion_matrixes[best_k-1]
print("best K is " + str(best_k))
print("accuracy: " + str(best_accuracy))
print("confusion matrix: (rows are real class and columns are predicted)")
for row in best_confusion_matrix.keys():
    print('\t' + row + ': ' + str(best_confusion_matrix[row]))

best K is 1
accuracy: 0.9600000000000002
confusion matrix: (rows are real class and columns are predicted)
	Iris-setosa: {'Iris-setosa': 50, 'Iris-versicolor': 0, 'Iris-virginica': 0}
	Iris-versicolor: {'Iris-setosa': 0, 'Iris-versicolor': 47, 'Iris-virginica': 3}
	Iris-virginica: {'Iris-setosa': 0, 'Iris-versicolor': 3, 'Iris-virginica': 47}
